In [2]:
import praw
import json

In [3]:
with open("config.json", "r") as jsonfile:
    data = json.load(jsonfile)  # Reading the config file
#     print("Config data read successful", data)

In [4]:
from collections import namedtuple  # Each element will be assigned a field
import sys
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession,SQLContext
# Allow us to order things in decscending order
from pyspark.sql.functions import desc

# create spark configuration
conf = SparkConf()
conf.setAppName("RedditStreamApp")
# create spark context with the above configuration
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
# create the Streaming Context from the above spark context with interval size 10 seconds
ssc = StreamingContext(sc, 10)
# setting a checkpoint to allow RDD recovery
#####ssc.checkpoint("checkpoint_RedditApp")
# read data from port 5590
# socket_stream = ssc.socketTextStream("localhost", 5590)


# sc = SparkContext() # Establish an entry point of Apache Spark functionality.
# ssc = StreamingContext(sc, 10) # Create streaming context with intervals of 10 seconds.
sqlContext = SQLContext(sc) # Initialising Spark SQL functionalities.

socket_stream = ssc.socketTextStream("127.0.0.1", 5590)


C:\Users\marke\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyspark\streaming\context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(
C:\Users\marke\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyspark\sql\context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
### Modify the code here to match your logic in reddit_producer.py
#**********************
lines = socket_stream.window(20)
fields = ("author",'date','time')
RedditComment = namedtuple('RedditComment', fields)

In [6]:
def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']


def test_content(string):
    print(string)
    return eval(string)


def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        # Get spark sql singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)
        ###################################################
        #### modify the code below to match your logic ####
        #### from the reddit_producer.py script        ####
        ###################################################

        
        # modify the mapping to account for data dictionary now being passed to it
        row_rdd = rdd.map(lambda rec: RedditComment(rec['author'],rec['date'],rec['time']))
        comments_df = sql_context.createDataFrame(row_rdd)
        comments_df.registerTempTable("comments")
        comments_df_sql = sql_context.sql(
            "select * from comments limit 10")
        comments_df_sql.show()

    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)



(lines.map(lambda text: test_content(text)).foreachRDD(process_rdd))

In [7]:
ssc.start()
# ssc.awaitTermination()  

----------- 2023-12-01 17:49:30 -----------
Error: <class 'ValueError'>
----------- 2023-12-01 17:49:40 -----------
Error: <class 'ValueError'>
----------- 2023-12-01 17:49:50 -----------


C:\Users\marke\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyspark\sql\dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+---------+----------+--------+
|   author|      date|    time|
+---------+----------+--------+
| 52dztg9j|2023-12-01|17:41:02|
| crafnp46|2023-12-01|17:41:15|
| 78o5tdf5|2023-12-01|17:41:19|
| 30x1absb|2023-12-01|17:41:21|
|o0qul26v4|2023-12-01|17:41:22|
| 10bgz8j4|2023-12-01|17:41:23|
|j2z7l6mgc|2023-12-01|17:41:23|
|    zn431|2023-12-01|17:41:25|
| u25c4b0s|2023-12-01|17:41:27|
| w14dew4a|2023-12-01|17:41:42|
+---------+----------+--------+

----------- 2023-12-01 17:50:00 -----------
+---------+----------+--------+
|   author|      date|    time|
+---------+----------+--------+
| 52dztg9j|2023-12-01|17:41:02|
| crafnp46|2023-12-01|17:41:15|
| 78o5tdf5|2023-12-01|17:41:19|
| 30x1absb|2023-12-01|17:41:21|
|o0qul26v4|2023-12-01|17:41:22|
| 10bgz8j4|2023-12-01|17:41:23|
|j2z7l6mgc|2023-12-01|17:41:23|
|    zn431|2023-12-01|17:41:25|
| u25c4b0s|2023-12-01|17:41:27|
| w14dew4a|2023-12-01|17:41:42|
+---------+----------+--------+

----------- 2023-12-01 17:50:10 -----------
+---------+---

In [8]:
ssc.stop()

Error: <class 'py4j.protocol.Py4JJavaError'>
